In [6]:
pip install tqdm


  Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
You should consider upgrading via the '/Users/himanshi/.pyenv/versions/3.8.13/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
from torchvision.datasets import ImageFolder
import os
import shutil
import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
from zipfile import ZipFile
import os

# Path to the uploaded zip files
zip_path_360 = '360 Rocks.zip'
zip_path_120 = '120 Rocks.zip'

# Directory to extract the contents
extract_dir_360 = '360 Rocks'
extract_dir_120 = '120 Rocks'

# Extract the files
with ZipFile(zip_path_360, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_360)

with ZipFile(zip_path_120, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_120)

# List the first few files in each directory to understand their structure
files_360 = os.listdir(extract_dir_360)[:10]
files_120 = os.listdir(extract_dir_120)[:10]

files_360, files_120


(['I_Diorite_06.jpg',
  'I_Diorite_12.jpg',
  'S_Breccia_09.jpg',
  'I_Obsidian_03.jpg',
  'I_Basalt_01.jpg',
  'M_Quartzite_05.jpg',
  'M_Quartzite_11.jpg',
  'I_Pumice_07.jpg',
  'S_Sandstone_03.jpg',
  'S_Bituminous Coal_06.jpg'],
 ['I_Diorite_2_120.jpg',
  'S_Rock Gypsum_1_120.jpg',
  'S_Conglomerate_2_120.jpg',
  'M_Slate_1_120.jpg',
  'M_Migmatite_2_120.jpg',
  'S_Dolomite_2_120.jpg',
  'M_Hornfels_1_120.jpg',
  'M_Schist_3_120.jpg',
  'I_Andesite_1_120.jpg',
  'I_Pegmatite_4_120.jpg'])

In [9]:
from collections import defaultdict
import shutil

# Directory for organized training and validation datasets
training_dir = 'Training Dataset'
validation_dir = 'Validation Dataset'

# Create directories if they do not exist
os.makedirs(training_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Function to group images by rock type
def group_images_by_rock_type(files, directory):
    rock_groups = defaultdict(list)
    for file in files:
        rock_type = file.split('_')[1]  # Assuming format is {Type}_{RockType}_{Number}.jpg
        rock_groups[rock_type].append(os.path.join(directory, file))
    return rock_groups

# List all files in the '360 Rocks' directory
all_files_360 = os.listdir(extract_dir_360)
rock_groups_360 = group_images_by_rock_type(all_files_360, extract_dir_360)

# Select 12 images per rock type for the training dataset
for rock_type, file_list in rock_groups_360.items():
    selected_files = file_list[:12]  # Take the first 12 images
    rock_type_dir = os.path.join(training_dir, rock_type)
    os.makedirs(rock_type_dir, exist_ok=True)
    for file_path in selected_files:
        shutil.copy(file_path, rock_type_dir)

# List the first few directories to confirm creation and population
os.listdir(training_dir)[:5]  # Display some of the rock type directories


['Basalt', 'Migmatite', 'Marble', 'Micrite', 'Granite']

In [10]:
# List all files in the '120 Rocks' directory
all_files_120 = os.listdir(extract_dir_120)
rock_groups_120 = group_images_by_rock_type(all_files_120, extract_dir_120)

# Select 4 images per rock type for the validation dataset
for rock_type, file_list in rock_groups_120.items():
    selected_files = file_list[:4]  # Take the first 4 images
    rock_type_dir = os.path.join(validation_dir, rock_type)
    os.makedirs(rock_type_dir, exist_ok=True)
    for file_path in selected_files:
        shutil.copy(file_path, rock_type_dir)

# List the first few directories to confirm creation and population
os.listdir(validation_dir)[:5]  # Display some of the rock type directories


['Basalt', 'Migmatite', 'Marble', 'Micrite', 'Granite']

In [11]:
import os
import random
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image

# Define more robust transformations with a wider range of randomness
data_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=(0, 90)),  # Wider range of rotation
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of horizontal flip
    transforms.RandomVerticalFlip(p=0.5),  # 50% chance of vertical flip
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0)),  # Random crops
    transforms.ToTensor(),
])

# Assuming train_folder is the path to the training dataset
train_folder = 'Training Dataset'  # Replace with your actual path to the training data
train_dataset = ImageFolder(train_folder, transform=data_transforms)

# Updated function to generate and save varied augmentations
def save_augmented_images(dataset, save_folder, num_augmented_images):
    os.makedirs(save_folder, exist_ok=True)
    total_images = 0

    for i, (image, label) in enumerate(dataset):
        category_folder = os.path.join(save_folder, dataset.classes[label])
        os.makedirs(category_folder, exist_ok=True)

        for j in range(num_augmented_images):
            # Apply additional transformations for each augmentation
            extra_transforms = transforms.Compose([
                transforms.RandomRotation(degrees=random.randint(0, 90)),
                transforms.ColorJitter(
                    brightness=random.uniform(0.1, 0.3),
                    contrast=random.uniform(0.1, 0.3),
                    saturation=random.uniform(0.1, 0.3),
                    hue=random.uniform(0.1, 0.3),
                ),
                transforms.ToPILImage(),  # Convert tensor to PIL Image for saving
            ])
            augmented_image = extra_transforms(image)  # Apply the extra transformations
            augmented_image_path = os.path.join(category_folder, f"augmented_{i}_{j}.jpeg")
            augmented_image.save(augmented_image_path)  # Save the image
            total_images += 1

    print(f"Total augmented images created: {total_images}")

# Path where augmented images will be saved
save_augmented_images_path = "train_aug"  # Adjust as needed
save_augmented_images(train_dataset, save_augmented_images_path, 10)


Total augmented images created: 3600


In [13]:
from torch.utils.data import DataLoader
train_folder = 'train_aug'
val_folder = "Validation Dataset"

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train = ImageFolder(train_folder, transform=transform)
val = ImageFolder(val_folder, transform=transform)

train_loader = DataLoader(train, batch_size=512, shuffle=True)
val_loader = DataLoader(val, batch_size=512, shuffle=False)







In [14]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import models
import tqdm

# Assuming 'device', 'train_loader', 'val_loader' are already defined

# Load pre-trained VGG16 model
model = torchvision.models.vgg16(pretrained=True)

# Freeze the parameters in the feature extractor part
for param in model.features.parameters():
    param.requires_grad = False

# Redefining the classifier part of VGG16
model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(25088, 4096),     # Original VGG first classifier layer size
    nn.ReLU(),
    nn.Dropout(0.5),            # Dropout for regularization
    nn.Linear(4096, 1024),      # Additional dense layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 8),         # Narrowing down before final layer
    nn.ReLU(),
    nn.Linear(8, 30),           # Final layer with 30 outputs
    nn.Softmax(dim=1)           # Softmax at the end for classification
)

# Move model to the appropriate device
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer (Optimize only the classifier parameters, feature layers are frozen)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    losses = []
    for batch_idx, (data, target) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    scheduler.step()  # Adjust learning rate
    
    avg_loss = sum(losses) / len(losses)
    print(f'Epoch - {epoch + 1}, Loss - {avg_loss:.4f}')

# Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the validation images: {accuracy:.2f}%')

# Save the trained model weights
torch.save(model.state_dict(), 'model_vgg16.pth')


/Users/himanshi/.pyenv/versions/3.8.13/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/himanshi/.pyenv/versions/3.8.13/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 8/8 [01:31<00:00, 11.49s/it]


Epoch - 1, Loss - 3.3970


100%|██████████| 8/8 [01:30<00:00, 11.26s/it]


Epoch - 2, Loss - 3.3660


 50%|█████     | 4/8 [00:50<00:49, 12.38s/it]